# COUNT DEPLOYED BY NANDINI LOKESH REDDY @MIND-LABS 

In [1]:
!nvidia-smi

Fri Apr 19 12:28:59 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   50C    P8    18W / 390W |      6MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/home/pratishthit/Desktop/Count


## Download video

In [3]:
SOURCE_VIDEO_PATH = "./fish-video1.avi"


In [4]:
!pip install ultralytics

In [4]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.29 🚀 Python-3.9.13 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080, 12045MiB)
Setup complete ✅ (16 CPUs, 31.2 GB RAM, 377.6/915.3 GB disk)


In [6]:
!pip install supervision

In [5]:
from IPython import display
display.clear_output()

import supervision as sv
print("supervision.__version__:", sv.__version__)

supervision.__version__: 0.19.0


In [36]:
import supervision as sv
import numpy as np

## Load pre-trained YOLOv8 model

In [6]:
MODEL = "best.pt"

In [7]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

YOLOv8m-seg summary (fused): 245 layers, 27226437 parameters, 0 gradients, 110.0 GFLOPs


## Predict and annotate single frame

In [8]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names

# class_ids of interest - car, motorcycle, bus and truck
selected_classes = [0, 1, 2, 3, 4, 5, 6]

In [9]:
CLASS_NAMES_DICT

{0: 'flounder-witch',
 1: 'flounder-witch-bottom',
 2: 'haddock',
 3: 'hake-red',
 4: 'hake-silver',
 5: 'pollock',
 6: 'redfish'}

### NORMAL COUNT USING IN AND OUT  --> OUTPUT: CHECK-1


In [26]:
# settings
LINE_START = sv.Point(750,150)
LINE_END = sv.Point(750, 1500)

TARGET_VIDEO_PATH = "./check1.mp4"

In [27]:
sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

VideoInfo(width=2048, height=1536, fps=15, total_frames=4544)

In [28]:
import cv2# create BYTETracker instance
byte_tracker = sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=30)

# create VideoInfo instance
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# create frame generator
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

# create LineZone instance, it is previously called LineCounter class
line_zone = sv.LineZone(start=LINE_START, end=LINE_END)

# create instance of BoxAnnotator
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)

# create instance of TraceAnnotator
trace_annotator = sv.TraceAnnotator(thickness=4, trace_length=50)

# create LineZoneAnnotator instance, it is previously called LineCounterAnnotator class
line_zone_annotator = sv.LineZoneAnnotator(thickness=4, text_thickness=4, text_scale=2)

# Define a dictionary to store class counts
class_counts = {}

# Define a dictionary to store unique tracking IDs for each class
class_tracking_ids = {class_name: set() for class_id, class_name in model.model.names.items()}

# define call back function to be used in video processing
def callback(frame: np.ndarray, index:int) -> np.ndarray:
    global class_counts, class_tracking_ids
    # model prediction on single frame and conversion to supervision Detections
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    # only consider class id from selected_classes define above
    detections = detections[np.isin(detections.class_id, selected_classes)]
    # tracking detections
    detections = byte_tracker.update_with_detections(detections)

    # Update class counts based on tracking IDs
    for class_id, tracker_id in zip(detections.class_id, detections.tracker_id):
        class_name = model.model.names[class_id]
        class_tracking_ids[class_name].add(tracker_id)

    labels = [
        f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
        for confidence, class_id, tracker_id
        in zip(detections.confidence, detections.class_id, detections.tracker_id)
    ]
    annotated_frame = trace_annotator.annotate(
        scene=frame.copy(),
        detections=detections
    )
    annotated_frame=box_annotator.annotate(
        scene=annotated_frame,
        detections=detections,
        labels=labels)

    # Count unique tracking IDs for each class
    class_counts = {class_name: len(tracking_ids) for class_name, tracking_ids in class_tracking_ids.items()}

    # # Add class counts in a text box at the top right corner of the video
    # for idx, (class_name, count) in enumerate(class_counts.items()):
    #     text = f"{class_name}: {count}"
    #     text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
    #     text_x = annotated_frame.shape[1] - 20 - text_size[0]
    #     text_y = 40 + idx * (text_size[1] + 20)
    #     cv2.putText(annotated_frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

    # update line counter
    line_zone.trigger(detections)
    # return frame with box and line annotated result
    return line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)

# process the whole video
sv.process_video(
    source_path = SOURCE_VIDEO_PATH,
    target_path = TARGET_VIDEO_PATH,
    callback=callback
)

# Print class counts after video processing
print("Class Counts:")
for class_name, count in class_counts.items():
    print(f"{class_name}: {count}")

SupervisionWarnings: `track_buffer` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'lost_track_buffer' instead.
SupervisionWarnings: `track_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'track_activation_threshold' instead.
SupervisionWarnings: `match_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'minimum_matching_threshold' instead.
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


Class Counts:
flounder-witch: 42
flounder-witch-bottom: 35
haddock: 10
hake-red: 55
hake-silver: 0
pollock: 9
redfish: 61


#### WITHOUT IN AND OUT" JUST WITH THE CLASS NAMES: WORKS WELL --> OUTPUT IN CHECK-2

In [24]:

from typing import Dict, Iterable, Optional, Tuple

import cv2
import numpy as np

from supervision.detection.core import Detections
from supervision.draw.color import Color
from supervision.draw.utils import draw_text
from supervision.geometry.core import Point, Position, Vector


class LineZone:


    def __init__(
        self,
        start: Point,
        end: Point,
        triggering_anchors: Iterable[Position] = (
            Position.TOP_LEFT,
            Position.TOP_RIGHT,
            Position.BOTTOM_LEFT,
            Position.BOTTOM_RIGHT,
        ),
    ):

        self.vector = Vector(start=start, end=end)
        self.limits = self.calculate_region_of_interest_limits(vector=self.vector)
        self.tracker_state: Dict[str, bool] = {}
        self.in_count: int = 0
        self.out_count: int = 0
        self.triggering_anchors = triggering_anchors

    @staticmethod
    def calculate_region_of_interest_limits(vector: Vector) -> Tuple[Vector, Vector]:
        magnitude = vector.magnitude

        if magnitude == 0:
            raise ValueError("The magnitude of the vector cannot be zero.")

        delta_x = vector.end.x - vector.start.x
        delta_y = vector.end.y - vector.start.y

        unit_vector_x = delta_x / magnitude
        unit_vector_y = delta_y / magnitude

        perpendicular_vector_x = -unit_vector_y
        perpendicular_vector_y = unit_vector_x

        start_region_limit = Vector(
            start=vector.start,
            end=Point(
                x=vector.start.x + perpendicular_vector_x,
                y=vector.start.y + perpendicular_vector_y,
            ),
        )
        end_region_limit = Vector(
            start=vector.end,
            end=Point(
                x=vector.end.x - perpendicular_vector_x,
                y=vector.end.y - perpendicular_vector_y,
            ),
        )
        return start_region_limit, end_region_limit

    @staticmethod
    def is_point_in_limits(point: Point, limits: Tuple[Vector, Vector]) -> bool:
        cross_product_1 = limits[0].cross_product(point)
        cross_product_2 = limits[1].cross_product(point)
        return (cross_product_1 > 0) == (cross_product_2 > 0)

    def trigger(self, detections: Detections) -> Tuple[np.ndarray, np.ndarray]:

        crossed_in = np.full(len(detections), False)
        crossed_out = np.full(len(detections), False)

        if len(detections) == 0:
            return crossed_in, crossed_out

        all_anchors = np.array(
            [
                detections.get_anchors_coordinates(anchor)
                for anchor in self.triggering_anchors
            ]
        )

        for i, tracker_id in enumerate(detections.tracker_id):
            if tracker_id is None:
                continue

            box_anchors = [Point(x=x, y=y) for x, y in all_anchors[:, i, :]]

            in_limits = all(
                [
                    self.is_point_in_limits(point=anchor, limits=self.limits)
                    for anchor in box_anchors
                ]
            )

            if not in_limits:
                continue

            triggers = [
                self.vector.cross_product(point=anchor) < 0 for anchor in box_anchors
            ]

            if len(set(triggers)) == 2:
                continue

            tracker_state = triggers[0]

            if tracker_id not in self.tracker_state:
                self.tracker_state[tracker_id] = tracker_state
                continue

            if self.tracker_state.get(tracker_id) == tracker_state:
                continue

            self.tracker_state[tracker_id] = tracker_state
            if tracker_state:
                self.in_count += 1
                crossed_in[i] = True
            else:
                self.out_count += 1
                crossed_out[i] = True

        return crossed_in, crossed_out


class LineZoneAnnotator:
    def __init__(
        self,
        thickness: float = 2,
        color: Color = Color.WHITE,
        text_thickness: float = 2,
        text_color: Color = Color.BLACK,
        text_scale: float = 0.5,
        text_offset: float = 1.5,
        text_padding: int = 10,
        custom_in_text: Optional[str] = None,
        custom_out_text: Optional[str] = None,
        display_in_count: bool = True,
        display_out_count: bool = True,
    ):

        self.thickness: float = thickness
        self.color: Color = color
        self.text_thickness: float = text_thickness
        self.text_color: Color = text_color
        self.text_scale: float = text_scale
        self.text_offset: float = text_offset
        self.text_padding: int = text_padding
        self.custom_in_text: str = custom_in_text
        self.custom_out_text: str = custom_out_text
        self.display_in_count: bool = display_in_count
        self.display_out_count: bool = display_out_count

    def _annotate_count(
        self,
        frame: np.ndarray,
        center_text_anchor: Point,
        text: str,
        is_in_count: bool,
    ) -> None:

        _, text_height = cv2.getTextSize(
            text, cv2.FONT_HERSHEY_SIMPLEX, self.text_scale, self.text_thickness
        )[0]

        if is_in_count:
            center_text_anchor.y -= int(self.text_offset * text_height)
        else:
            center_text_anchor.y += int(self.text_offset * text_height)

        draw_text(
            scene=frame,
            text=text,
            text_anchor=center_text_anchor,
            text_color=self.text_color,
            text_scale=self.text_scale,
            text_thickness=self.text_thickness,
            text_padding=self.text_padding,
            background_color=self.color,
        )
    def annotate(self, frame: np.ndarray, line_counter: LineZone) -> np.ndarray:

        cv2.line(
            frame,
            line_counter.vector.start.as_xy_int_tuple(),
            line_counter.vector.end.as_xy_int_tuple(),
            self.color.as_bgr(),
            self.thickness,
            lineType=cv2.LINE_AA,
            shift=0,
        )
        cv2.circle(
            frame,
            line_counter.vector.start.as_xy_int_tuple(),
            radius=5,
            color=self.text_color.as_bgr(),
            thickness=-1,
            lineType=cv2.LINE_AA,
        )
        cv2.circle(
            frame,
            line_counter.vector.end.as_xy_int_tuple(),
            radius=5,
            color=self.text_color.as_bgr(),
            thickness=-1,
            lineType=cv2.LINE_AA,
        )

        return frame


In [22]:
TARGET_VIDEO_PATH = "./check2.mp4"

In [25]:
import cv2# create BYTETracker instance
byte_tracker = sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=30)

# create VideoInfo instance
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# create frame generator
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

# create LineZone instance, it is previously called LineCounter class
line_zone = LineZone(start=LINE_START, end=LINE_END)

# create instance of BoxAnnotator
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)

# create instance of TraceAnnotator
trace_annotator = sv.TraceAnnotator(thickness=4, trace_length=50)

# create LineZoneAnnotator instance, it is previously called LineCounterAnnotator class
line_zone_annotator = LineZoneAnnotator(thickness=4, text_thickness=4, text_scale=2)

# Define a dictionary to store class counts
class_counts = {}

# Define a dictionary to store unique tracking IDs for each class
class_tracking_ids = {class_name: set() for class_id, class_name in model.model.names.items()}

# define call back function to be used in video processing
def callback(frame: np.ndarray, index:int) -> np.ndarray:
    global class_counts, class_tracking_ids
    # model prediction on single frame and conversion to supervision Detections
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    # only consider class id from selected_classes define above
    detections = detections[np.isin(detections.class_id, selected_classes)]
    # tracking detections
    detections = byte_tracker.update_with_detections(detections)

    # Update class counts based on tracking IDs
    for class_id, tracker_id in zip(detections.class_id, detections.tracker_id):
        class_name = model.model.names[class_id]
        class_tracking_ids[class_name].add(tracker_id)

    labels = [
        f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
        for confidence, class_id, tracker_id
        in zip(detections.confidence, detections.class_id, detections.tracker_id)
    ]
    annotated_frame = trace_annotator.annotate(
        scene=frame.copy(),
        detections=detections
    )
    annotated_frame=box_annotator.annotate(
        scene=annotated_frame,
        detections=detections,
        labels=labels)

    # Count unique tracking IDs for each class
    class_counts = {class_name: len(tracking_ids) for class_name, tracking_ids in class_tracking_ids.items()}

    # Add class counts in a text box at the top right corner of the video
    for idx, (class_name, count) in enumerate(class_counts.items()):
        text = f"{class_name}: {count}"
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
        text_x = annotated_frame.shape[1] - 20 - text_size[0]
        text_y = 40 + idx * (text_size[1] + 20)
        cv2.putText(annotated_frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

    # update line counter
    line_zone.trigger(detections)
    # return frame with box and line annotated result
    return line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)

# process the whole video
sv.process_video(
    source_path = SOURCE_VIDEO_PATH,
    target_path = TARGET_VIDEO_PATH,
    callback=callback
)

# Print class counts after video processing
print("Class Counts:")
for class_name, count in class_counts.items():
    print(f"{class_name}: {count}")

SupervisionWarnings: `track_buffer` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'lost_track_buffer' instead.
SupervisionWarnings: `track_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'track_activation_threshold' instead.
SupervisionWarnings: `match_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'minimum_matching_threshold' instead.
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


Class Counts:
flounder-witch: 42
flounder-witch-bottom: 35
haddock: 10
hake-red: 55
hake-silver: 0
pollock: 9
redfish: 61


### SAVING ONE FRAME FOR EACH FISH {WHEN BOUNDING BOX MEETS THE SPECIFIED LINE}  --> OUTPUT : SAVED_FRAMESCHECK-2

In [34]:
import os

# Define the folder where frames will be saved
frames_folder = "saved_framesCHECK-2"

# Create the folder if it doesn't exist
if not os.path.exists(frames_folder):
    os.makedirs(frames_folder)

# Define a set to keep track of frames where fish bounding boxes cross the line
frames_to_save = set()

# define call back function to be used in video processing
def callback(frame: np.ndarray, index:int):
    global class_counts, class_tracking_ids, frames_to_save
    # model prediction on single frame and conversion to supervision Detections
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    # only consider class id from selected_classes define above
    detections = detections[np.isin(detections.class_id, selected_classes)]
    # tracking detections
    detections = byte_tracker.update_with_detections(detections)

    # Update class counts based on tracking IDs
    for class_id, tracker_id in zip(detections.class_id, detections.tracker_id):
        class_name = model.model.names[class_id]
        class_tracking_ids[class_name].add(tracker_id)

    labels = [
        f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
        for confidence, class_id, tracker_id
        in zip(detections.confidence, detections.class_id, detections.tracker_id)
    ]
    annotated_frame = trace_annotator.annotate(
        scene=frame.copy(),
        detections=detections
    )
    annotated_frame=box_annotator.annotate(
        scene=annotated_frame,
        detections=detections,
        labels=labels)

    # Count unique tracking IDs for each class
    class_counts = {class_name: len(tracking_ids) for class_name, tracking_ids in class_tracking_ids.items()}

    # Add class counts in a text box at the top right corner of the video
    for idx, (class_name, count) in enumerate(class_counts.items()):
        text = f"{class_name}: {count}"
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
        text_x = annotated_frame.shape[1] - 20 - text_size[0]
        text_y = 40 + idx * (text_size[1] + 20)
        cv2.putText(annotated_frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

    # update line counter
    crossed_in, crossed_out = line_zone.trigger(detections)
    
    # Save frames where fish bounding boxes cross the line for the first time
    for tracker_id, crossed, class_id in zip(detections.tracker_id, crossed_in, detections.class_id):
        if tracker_id is not None and crossed:
            if tracker_id not in frames_to_save:
                frames_to_save.add(tracker_id)
                class_name = model.model.names[class_id]
                # Save the frame in the folder with class name and tracker ID in the filename
                frame_path = os.path.join(frames_folder, f'frame_{class_name}_{tracker_id}.jpg')
                cv2.imwrite(frame_path, frame)


    # return frame with box and line annotated result
    return line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)


In [35]:
import os
import cv2

import numpy as np


# Create the folder if it doesn't exist
if not os.path.exists(frames_folder):
    os.makedirs(frames_folder)


# Call the process_video function to process the source video
sv.process_video(
    source_path=SOURCE_VIDEO_PATH,
    target_path=None,  # Setting target_path to None to avoid saving the video
    callback=callback
)


[ERROR:0@3084.285] global cap.cpp:643 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.9.0) /io/opencv/modules/videoio/src/cap_images.cpp:216: error: (-215:Assertion failed) !filename.empty() in function 'icvExtractPattern'


